In [14]:
import numpy as np
import torch
import torch.distributions as dis

In [12]:
def relative_stat(estimate, se, truth):
	"""Return relative """
	return (estimate - truth) / truth, se / truth


def experiment_step(d_loc, T):
	p = dis.Normal(loc=0, scale=1)
	q = dis.Normal(loc=d_loc, scale=1)
	truth = dis.kl_divergence(q, p)

	x = q.sample(sample_shape=(T,))
	logr = p.log_prob(x) - q.log_prob(x)

	k1 = -logr
	k2 = logr ** 2 / 2
	k3 = (logr.exp() - 1) - logr

	relative_bias_list, relative_se_list = [], []
	for k in (k1, k2, k3):
		relative_bias, relative_se = relative_stat(k.mean(), k.std(), truth)
		relative_bias_list.append(float(relative_bias))
		relative_se_list.append(float(relative_se))
	
	return relative_bias_list, relative_se_list

In [13]:
experiment_step(0.1, 50)

([2.236786365509033, -0.030158767476677895, -0.038029663264751434],
 [19.625577926635742, 1.287672996520996, 1.2843981981277466])

In [ ]:
rel_biaes = []
rel_ses = []
for d in np.linspace(0.5, 5, 37):
	_rel_bias, _rel_se = experiment_step(d, 50)
	rel_biaes.append(_rel_bias)
	rel_ses.append(_rel_se)

exp_data = np.column_stack([np.linspace(0.5, 5, 37), np.array(rel_biaes), np.array(rel_ses)])